In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def simulate_ball_on_plate(friction=0.98, tsx=0.0, tsy=0.0, Kp=4.0, Ki=2.0, Kd=1.0):
    """
    Simulate a ball on a plate using a PID controller to control the position of the ball.

    Parameters:
    - friction: Friction coefficient of the plate.
    - tsx: Target position in x direction.
    - tsy: Target position in y direction.
    - Kp: Proportional gain for PID controller.
    - Ki: Integral gain for PID controller.
    - Kd: Derivative gain for PID controller.

    The simulation calculates the position and angles of the ball on the plate over a number of steps,
    applying the PID controller to adjust the roll and pitch angles based on the target positions.
    """
    # Initial conditions
    roll = 0.0 # Current roll angle in degrees
    pitch = 0.0 # Current pitch angle in degrees
    sx = 0.2 # Current Position in meters
    sy = 0.2 # Current Position in meters
    vx = 0.0 # Current velocity in m/s
    vy = 0.0 # Current velocity in m/s
    g = 9.81 # Gravity g in m/s^2
    delta_t = 0.1 # Interval in seconds (s)
    max_angle = np.radians(4) # Max angels
    steps = 100 # Steps

    # State variables for PID controller
    # Integral and previous error terms for PID control
    integral_x = 0.0
    integral_y = 0.0
    prev_error_x = 0.0
    prev_error_y = 0.0

    # Storage for positions and angles
    positions_x = []
    positions_y = []
    angles_x = []
    angles_y = []
    
    # Iterate over the number of steps
    for _ in range(steps):
        # Calculate accelerate for x
        roll_theta = roll
        ax = (3/5) * g * np.sin(roll_theta)

        # Calculate accelerate for y
        pitch_theta = pitch
        ay = (3/5) * g * np.sin(pitch_theta)

        # Calculcate velocity for x
        vx = vx + ax * delta_t

        # Calculcate velocity for y
        vy = vy + ay * delta_t

        # friction
        vx *=friction
        vy *=friction

        # Calculcate position for x
        sx = sx + vx * delta_t + (1/2) * ax * (delta_t * delta_t)

        # Calculcate position for y
        sy = sy + vy * delta_t + (1/2) * ay * (delta_t * delta_t)

        # PID for x
        ex = tsx - sx
        integral_x += ex * delta_t
        derivative_x = (ex - prev_error_x) / delta_t
        prev_error_x = ex
        ux = Kp * ex + Ki * integral_x + Kd * derivative_x

        # Set roll
        roll = np.clip(ux, -max_angle, max_angle)

        positions_x.append(sx)
        angles_x.append(np.degrees(roll))

        # PID for y
        ey = tsy - sy
        integral_y += ey * delta_t
        derivative_y = (ey - prev_error_y) / delta_t
        prev_error_y = ey
        uy = Kp * ey + Ki * integral_y + Kd * derivative_y

        # # Set pitch
        pitch = np.clip(uy, -max_angle, max_angle)

        positions_y.append(sy)
        angles_y.append(np.degrees(pitch))

    # Plot
    fig, ax = plt.subplots(2, 2, figsize=(10, 8))

    ax[0, 0].plot(positions_x)
    ax[0, 0].set(title='Position x',  ylabel='m')

    ax[0, 1].plot(angles_x)
    ax[0, 1].set(title='Rollwinkel',  ylabel='°')
    
    ax[1, 0].plot(positions_y)
    ax[1, 0].set(title='Position y',  ylabel='m', xlabel='Zeitschritt')
    
    ax[1, 1].plot(angles_y)
    ax[1, 1].set(title='Pitchwinkel', ylabel='°', xlabel='Zeitschritt')

    plt.show()

# Interactive widget to control the simulation parameters
interact(simulate_ball_on_plate,
         friction=FloatSlider(min=0, max=1, step=0.005, value=0.98, description='Friction'),
         tsx=FloatSlider(min=-1, max=1, step=0.05, value=0.0, description='Target X'),
         tsy=FloatSlider(min=-1, max=1, step=0.05, value=0.0, description='Target Y'),
         Kp=FloatSlider(min=0, max=10, step=0.1, value=1.0, description='Kp'),
         Ki=FloatSlider(min=0, max=10, step=0.1, value=0.0, description='Ki'),
         Kd=FloatSlider(min=0, max=10, step=0.1, value=0.5, description='Kd'))

interactive(children=(FloatSlider(value=0.98, description='Friction', max=1.0, step=0.005), FloatSlider(value=…

<function __main__.simulate_ball_on_plate(friction=0.98, tsx=0.0, tsy=0.0, Kp=4.0, Ki=2.0, Kd=1.0)>